# Ernesto Antonio Reyes Ramírez

# Examen Parcial

# Optimización

### Implementar el algoritmo de región de confianza basado en el Algoritmo 1. 

In [7]:
#Librerias
import numpy as np
import math
import copy
import time

In [38]:
def find_lambda(B):
    """
    Encuentra el valor mínimo de lambda tal que B + lambda*I es positiva definida.

    Parameters:
        B (numpy.ndarray): matriz de entrada.

    Returns:
        lambda_min (float): valor mínimo de lambda tal que B + lambda*I es positiva definida.
    """
    eigenvals = np.linalg.eigvals(B)
    lambda_min = -np.min(eigenvals) + 1
    return lambda_min

In [31]:
def is_positive_semidefinite(A):
    """
    Determina si una matriz es semidefinida positiva.

    Parameters:
        A (numpy.ndarray): matriz de entrada.

    Returns:
        bool: True si A es semidefinida positiva, False en caso contrario.
    """
    eigenvals = np.linalg.eigvals(A)
    return np.all(eigenvals >= 0)

In [32]:
#Algoritmo 1
def alg1(fx,g,B,y0,Delta):
    y = y0
    m = B.shape[0]
    for n in range(0,100):
        A = B + y*np.eye(m)
        L = np.linalg.cholesky(A)
        z = -np.linalg.solve(L,g)
        p = np.linalg.solve(L.T,z)
        q = np.linalg.solve(L,p)
        
        y = y + ((np.linalg.norm(p)/np.linalg.norm(q))**2)*((np.linalg.norm(p)-Delta)/Delta)
    
    P = - np.linalg.inv(B + y*np.eye(m))@g
    return P

In [39]:
#Región de confianza con algoritmo 1
def Trust_region_alg1(f,grad_f,H_f,x0,Delta_0,Delta_max,eta,tol,ite):
    x = x0
    Delta = Delta_0
    for i in range(ite):
        g = grad_f(x)
        B = H_f(x)
        if is_positive_semidefinite(B) == True:
            p = -np.linalg.inv(B)@g
        else:
            y = find_lambda(B)
            p = alg1(f(x),g,B,y,Delta)
        rho = (f(x) - f(x + p)) / (-np.dot(g, p) - 0.5 * np.dot(p, np.dot(B, p)))
        if rho < eta:
            Delta *= 0.25
        else:
            if rho > 0.75 and np.linalg.norm(p) == Delta:
                Delta = min(2 * Delta, Delta_max)
            if rho > eta:
                x += p
        if np.linalg.norm(g) < tol:
            break
            
    return x,i

In [35]:
# Región de confianza con Dogleg
def Dogleg(x, fx, grad_fx, H_fx, Delta):
    pB = -np.linalg.solve(H_fx,grad_fx)
    if np.linalg.norm(pB) <= Delta:
        return pB
    
    pU = -((grad_fx.T@grad_fx)/(grad_fx.T@H_fx@grad_fx))*grad_fx
    
    if np.linalg.norm(pU) >= Delta:
        return (Delta / np.linalg.norm(grad_fx)) * grad_fx
    a = np.dot(pB - pU, pB - pU)
    b = 2 * np.dot(pU, pB - pU)
    c = np.dot(pU, pU) - Delta ** 2
    t = (-b + np.sqrt(b ** 2 - 4 * a * c)) / (2 * a)
    return pU + t * (pB - pU)
    
    
    
def Trust_region_Dogleg(f,grad_f,H_f,x0,Delta_0,Delta_max,eta,tol,ite):
    x = x0
    Delta = Delta_0
    for i in range(ite):
        g = grad_f(x)
        B = H_f(x)
        eps = 1e-6
        B += np.eye(B.shape[0]) * eps
        p = Dogleg(x, f(x), g, B, Delta)
        rho = (f(x) - f(x + p)) / (-np.dot(g, p) - 0.5 * np.dot(p, np.dot(B, p)))
        if rho < eta:
            Delta *= 0.25
        else:
            if rho > 0.75 and np.linalg.norm(p) == Delta:
                Delta = min(2 * Delta, Delta_max)
            if rho > eta:
                x += p
        if np.linalg.norm(g) < tol:
            break
            
    return x,i

# Experimentos

In [36]:
#Función Rosembrock 

def f_Rosenbrock(x):
    n = len(x)
    sum = 0
    for i in range(n-1):
        sum += 100*(x[i+1] - x[i]**2)**2 + (1 - x[i])**2
    return sum


def grad_rosenbrock(x):
    n = len(x)
    grad_f = np.zeros(n)
    grad_f[0] = 400*(x[0]**3) - 400*x[0]*x[1]+2*x[0]-2
    grad_f[n-1] = 200*x[n-1]-200*(x[n-2]**2) 
    
    for i in range(1,n-1):
        grad_f[i] = 400*(x[i]**3) + 202*x[i] - 400*x[i]*x[i+1]-200*(x[i-1]**2) - 2
    
    return grad_f

def H_rosenbrock(x):
    n = len(x)
    h_f = np.zeros((n,n))
    h_f[0,0] = 1200*(x[0]**2) - 400*x[1]+2
    h_f[0,1] = -400*x[0]
    h_f[n-1,n-2] = -400*x[n-2]
    h_f[n-1,n-1] = 200
    
    for i in range(1,n):
        h_f[i,i-1] = -400*x[i-1] 
    
    for i in range(1,n-1):
        h_f[i,i] = 1200*(x[i]**2) + 202 - 400*x[i+1]
        
    for i in range(0,n-1):
        h_f[i,i+1] = -400*x[i] 
        
    return h_f

In [51]:
# Región de confianza con Algoritmo 1, Función Rosenbrock
n = 2
m = 30
tim = 0
it = 0
f = 0

for i in range(m):
    x0 = np.random.uniform(-2,2,n) + 1
    inicio = time.time()
    xmin,k = Trust_region_alg1(f_Rosenbrock,grad_rosenbrock,H_rosenbrock,x0,1,10,0.25,1e-5,100)
    fin = time.time()
    tim = tim + (fin-inicio)
    it = it + k
    if k == 99:
        f = f+1

tim = tim/m
it = it/m
print("Método de región de confianza con Algoritmo 1")
print("Tiempo promedio: ", tim)
print("Iteraciones promedio: ", it)
print("Número de fallas: ", f)

Método de región de confianza con Algoritmo 1
Tiempo promedio:  0.009009083112080893
Iteraciones promedio:  89.73333333333333
Número de fallas:  27


In [52]:
# Región de confianza con Dogleg, Función Rosenbrock
n = 2
m = 30
tim = 0
it = 0
f = 0

for i in range(m):
    x0 = np.random.uniform(-2,2,n) + 1
    inicio = time.time()
    xmin,k = Trust_region_Dogleg(f_Rosenbrock,grad_rosenbrock,H_rosenbrock,x0,1,10,0.25,1e-8,100)
    fin = time.time()
    tim = tim + (fin-inicio)
    it = it + k
    if k == 99:
        f = f+1

tim = tim/m
it = it/m
print("Método de región de confianza con Dogleg")
print("Tiempo promedio: ", tim)
print("Iteraciones promedio: ", it)
print("Número de fallas: ", f)

Método de región de confianza con Dogleg
Tiempo promedio:  0.00082550048828125
Iteraciones promedio:  18.966666666666665
Número de fallas:  0


Conclusiones: Como podemos notar, el algoritmo 1 logra converger al mínimo pero también existen veces en las que no lo logra encontrar, esto es de esperarze ya que como se menciona en el libro este algoritmo no es de los mejores y suele fallar. El promedio de iteraciones fue algo alto, que es 90 iteraciones aproximádamente, tomando en cuenta que los que fallaron utilizaron las 100 iteraciones. Por otro lado, el algoritmo implementado con Dogleg no tuvo ninguna falla en las 30 iteraciones y logró converger demasiado rápido, en un promedio de 19 iteraciones. Con estos criterios en cuenta, concluimos que el algoritmo 1 funciona pero en todos los aspectos el Dogleg es mejor que él. 